In [2]:
import bs4 as bs
import pickle
import requests

def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
        
    return tickers

save_sp500_tickers()

[u'MMM',
 u'ABT',
 u'ABBV',
 u'ACN',
 u'ATVI',
 u'AYI',
 u'ADBE',
 u'AAP',
 u'AES',
 u'AET',
 u'AMG',
 u'AFL',
 u'A',
 u'APD',
 u'AKAM',
 u'ALK',
 u'ALB',
 u'AGN',
 u'LNT',
 u'ALXN',
 u'ALLE',
 u'ADS',
 u'ALL',
 u'GOOGL',
 u'GOOG',
 u'MO',
 u'AMZN',
 u'AEE',
 u'AAL',
 u'AEP',
 u'AXP',
 u'AIG',
 u'AMT',
 u'AWK',
 u'AMP',
 u'ABC',
 u'AME',
 u'AMGN',
 u'APH',
 u'APC',
 u'ADI',
 u'ANTM',
 u'AON',
 u'APA',
 u'AIV',
 u'AAPL',
 u'AMAT',
 u'ADM',
 u'ARNC',
 u'AJG',
 u'AIZ',
 u'T',
 u'ADSK',
 u'ADP',
 u'AN',
 u'AZO',
 u'AVB',
 u'AVY',
 u'BHI',
 u'BLL',
 u'BAC',
 u'BK',
 u'BCR',
 u'BAX',
 u'BBT',
 u'BDX',
 u'BBBY',
 u'BRK.B',
 u'BBY',
 u'BIIB',
 u'BLK',
 u'HRB',
 u'BA',
 u'BWA',
 u'BXP',
 u'BSX',
 u'BMY',
 u'AVGO',
 u'BF.B',
 u'CHRW',
 u'CA',
 u'COG',
 u'CPB',
 u'COF',
 u'CAH',
 u'HSIC',
 u'KMX',
 u'CCL',
 u'CAT',
 u'CBG',
 u'CBS',
 u'CELG',
 u'CNC',
 u'CNP',
 u'CTL',
 u'CERN',
 u'CF',
 u'SCHW',
 u'CHTR',
 u'CHK',
 u'CVX',
 u'CMG',
 u'CB',
 u'CHD',
 u'CI',
 u'XEC',
 u'CINF',
 u'CTAS',
 u'CSCO',
